import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Générer des données d'exemple (à remplacer par vos propres données)
def generate_example_data():
    image_size = 64
    image1 = np.random.rand(image_size, image_size, 3)
    image2 = np.random.rand(image_size, image_size, 3)

    # Créer un décalage aléatoire entre les deux images (simulant le recalage)
    shift = np.random.randint(-10, 10, size=(2,))
    image2_shifted = np.roll(image2, shift, axis=(0, 1))

    return image1, image2_shifted

# Générer des données d'entraînement
def generate_training_data(num_samples):
    data = [generate_example_data() for _ in range(num_samples)]
    x_train, y_train = zip(*data)
    return np.array(x_train), np.array(y_train)

# Modèle CNN simple pour le recalage des images
def create_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(np.prod(input_shape), activation='sigmoid'))  # Modification ici
    model.add(layers.Reshape(input_shape))  # Modification ici
    return model

# Hyperparamètres
num_samples = 1000
batch_size = 8
epochs = 10

# Générer des données d'entraînement
x_train, y_train = generate_training_data(num_samples)

# Créer le modèle
input_shape = x_train[0].shape
model = create_cnn_model(input_shape)

# Compiler le modèle
model.compile(optimizer=Adam(), loss='mse')

# Entraîner le modèle
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

# Sauvegarder le modèle (facultatif)
model.save('image_registration_model.h5')

# Utiliser le modèle pour recalibrer une nouvelle paire d'images
image1_test, image2_test_shifted = generate_example_data()
predicted_shift = model.predict(np.expand_dims(image1_test, axis=0))
image2_test_recalled = np.roll(image2_test_shifted, (predicted_shift[0][:2]).astype(int), axis=(0, 1))

# Afficher les résultats (à adapter en fonction de votre environnement)
import matplotlib.pyplot as plt

print(predicted_shift[0].shape)
plt.figure(figsize=(8, 4))

plt.subplot(1, 3, 1)
plt.title('Image 1')
plt.imshow(image1_test)

plt.subplot(1, 3, 2)
plt.title('Image 2 (décalée)')
plt.imshow(image2_test_shifted)

plt.subplot(1, 3, 3)
plt.title('Image 2 recalée (prédiction du modèle)')
plt.imshow(image2_test_recalled)

plt.show()

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Chemins vers vos images d'entraînement
image1_path = 'Documents/TDSI/PTI/20230323_191223_BRS_L_UP_11.369_00.jpg'
image2_path = 'Documents/TDSI/PTI/20230406_060421_BRS_L_UP_11.369_00.jpg'

# Fonction pour charger et prétraiter les images
def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(64, 64))  # Ajustez la taille selon vos besoins
    img_array = image.img_to_array(img)
    img_array /= 255.0  # Normalisation des valeurs de pixel
    return img_array

# Charger vos images d'entraînement
image1 = load_and_preprocess_image(image1_path)
image2 = load_and_preprocess_image(image2_path)

# Créer un décalage aléatoire entre les deux images (simulant le recalage)
shift = np.random.randint(-10, 10, size=(2,))
image2_shifted = np.roll(image2, shift, axis=(0, 1))

# Afficher les images d'entraînement
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title('Image 1')
plt.imshow(image1)

plt.subplot(1, 2, 2)
plt.title('Image 2 (décalée)')
plt.imshow(image2_shifted)
plt.show()

# Préparer les données d'entraînement
x_train = np.expand_dims(image1, axis=0)
y_train = np.expand_dims(image2_shifted, axis=0)

# Modèle CNN simple pour le recalage des images
def create_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2))  # Deux sorties pour les décalages en x et y
    return model

# Créer le modèle
input_shape = x_train.shape[1:]
model = create_cnn_model(input_shape)

# Compiler le modèle
model.compile(optimizer=Adam(), loss='mse')

# Entraîner le modèle
model.fit(x_train, y_train, epochs=10)

# Utiliser le modèle pour recalibrer la paire d'images
predicted_shift = model.predict(x_train)
image2_recalled = np.roll(image2_shifted, predicted_shift[0].astype(int), axis=(0, 1))

# Afficher les résultats
plt.figure(figsize=(8, 4))
plt.subplot(1, 3, 1)
plt.title('Image 1')
plt.imshow(image1)

plt.subplot(1, 3, 2)
plt.title('Image 2 (décalée)')
plt.imshow(image2_shifted)

plt.subplot(1, 3, 3)
plt.title('Image 2 recalée (prédiction du modèle)')
plt.imshow(image2_recalled)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'Documents/TDSI/PTI/20230323_191223_BRS_L_UP_11.369_00.jpg'